In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pathlib import Path

# ============================================================
# 0. Geometry builder from PCA input
# ============================================================
def build_geometry_from_pca(geom_input,
                            base_Lz=0.1,
                            base_R=0.005,
                            base_Nz=50,
                            base_Nr=25):
    """
    Map a PCA geometry vector (geom_input) to physical geometry parameters.

    Parameters
    ----------
    geom_input : array-like
        PCA geometry vector or other reduced representation of geometry.
        e.g. [PC1, PC2, ...]. You decide the mapping.
    base_Lz, base_R : float
        Baseline length and radius used in the original ROM.
    base_Nz, base_Nr : int
        Baseline axial and radial grid counts.

    Returns
    -------
    geom : dict
        {
          "Lz": ...,
          "R": ...,
          "Nz": ...,
          "Nr": ...,
          "dz": ...,
          "dr": ...,
          "z": 1D array of cell-center axial positions,
          "r": 1D array of cell-center radial positions,
        }
    """
    g = np.asarray(geom_input, float)

    # --- EXAMPLE SIMPLE MAPPING (you can change this!) ---
    #   PC1 scales axial length (±20%)
    #   PC2 scales radius      (±20%)
    #   grid resolution kept fixed here (but you could also change Nz/Nr)
    length_scale = 1.0 + 0.2 * (g[0] if g.size > 0 else 0.0)
    radius_scale = 1.0 + 0.2 * (g[1] if g.size > 1 else 0.0)

    Lz = base_Lz * length_scale
    R = base_R * radius_scale

    Nz = int(base_Nz)
    Nr = int(base_Nr)

    dz = Lz / Nz
    dr = R / Nr

    z = np.linspace(dz / 2, Lz - dz / 2, Nz)
    r = np.linspace(dr / 2, R - dr / 2, Nr)

    geom = dict(Lz=Lz, R=R, Nz=Nz, Nr=Nr, dz=dz, dr=dr, z=z, r=r)
    return geom


# ============================================================
# 1. Time setup (depends on geometry)
# ============================================================
def build_time_grid(geom, D=1e-7, T_total_min=150.0, safety=0.1):
    """
    Given geometry and diffusion coefficient, build dt, n_steps, and time array.

    safety: CFL-like factor (0.1 = 10% of diffusive stability limit).
    """
    dz, dr = geom["dz"], geom["dr"]
    dt = safety * min(dr**2, dz**2) / D
    n_steps = int(round(T_total_min * 60.0 / dt))
    time = np.arange(n_steps) * dt
    return time, dt, n_steps


# ============================================================
# 2. Experimental data (normalized flux)
# ============================================================
t_exp = np.array([
    3.2, 3.91667, 5.43333, 6.4, 7.55, 9.25, 10.88333, 13.0,
    14.8, 19.01667, 21.86667, 26.4, 29.0, 32.03333
]) * 60.0  # seconds

f_exp_norm = np.array([
    0.67836, 0.60243, 0.48638, 0.42766, 0.37502, 0.3163,
    0.27751, 0.23958, 0.21682, 0.17867, 0.16056, 0.13943,
    0.13018, 0.1208
])

delay = 0.0
D = 1e-7  # diffusion coefficient (kept constant here)


# ============================================================
# 3. Stub ROM – replace with your actual finite-volume solver
# ============================================================
def simulate_membrane_flux(P_t, geom, D, dt, n_steps):
    """
    Placeholder implementation of the ROM.
    In MATLAB this was `simulateMembraneFlux`.

    Parameters
    ----------
    P_t : array of shape (n_steps,)
        Time-dependent permeability / effective resistance.
    geom : dict
        Geometry dictionary from build_geometry_from_pca.
    D : float
        Diffusion coefficient.
    dt : float
        Time step [s].
    n_steps : int
        Number of time steps.

    Returns
    -------
    flux : array of shape (n_steps,)
        Normalized or absolute flux vs time.

    NOTE: This is a placeholder (flux ∝ P_t). Replace with your PDE solver.
    """
    flux = np.asarray(P_t, dtype=float).copy()
    return flux, None


# ============================================================
# 4. Cost functions (now geometry + time are passed in)
# ============================================================
def cost_exponential(params, time, geom,
                     t_exp, f_exp_norm,
                     D, dt, n_steps, delay):
    S0, P0, k, Pmin = params  # S0 kept for compatibility
    t_eff = np.maximum(0.0, time - delay)
    P_t = Pmin + (P0 - Pmin) * np.exp(-k * t_eff)
    flux, _ = simulate_membrane_flux(P_t, geom, D, dt, n_steps)
    flux_norm = flux / flux[0]
    flux_interp = np.interp(t_exp, time, flux_norm)
    return np.sum(((flux_interp - f_exp_norm) / f_exp_norm) ** 2)


def cost_powerlaw(params, time, geom,
                  t_exp, f_exp_norm,
                  D, dt, n_steps, delay):
    S0, A, n, Pmin = params
    t_eff = np.maximum(time - delay, 1.0)
    P_t = Pmin + (A - Pmin) * (t_eff ** (-n))
    flux, _ = simulate_membrane_flux(P_t, geom, D, dt, n_steps)
    flux_norm = flux / flux[0]
    flux_interp = np.interp(t_exp, time, flux_norm)
    return np.sum(((flux_interp - f_exp_norm) / f_exp_norm) ** 2)


def cost_logistic(params, time, geom,
                  t_exp, f_exp_norm,
                  D, dt, n_steps):
    S0, P0, k, tmid, Pmin = params
    P_t = Pmin + (P0 - Pmin) / (1.0 + np.exp(k * (time - tmid)))
    flux, _ = simulate_membrane_flux(P_t, geom, D, dt, n_steps)
    flux_norm = flux / flux[0]
    flux_interp = np.interp(t_exp, time, flux_norm)
    return np.sum(((flux_interp - f_exp_norm) / f_exp_norm) ** 2)


def cost_logistic_jmin(params, time, geom,
                       t_exp, f_exp_norm,
                       D, dt, n_steps):
    Jmin, P0, k, tmid = params
    P_t = Jmin + (P0 - Jmin) / (1.0 + np.exp(k * (time - tmid)))
    flux, _ = simulate_membrane_flux(P_t, geom, D, dt, n_steps)
    flux_norm = flux / flux[0]
    flux_interp = np.interp(t_exp, time, flux_norm)
    return np.sum(((flux_interp - f_exp_norm) / f_exp_norm) ** 2)


def cost_hybrid_jmin(params, time, geom,
                     t_exp, f_exp_norm,
                     D, dt, n_steps, delay):
    Jmin, P0, k1, k2 = params
    t_eff = np.maximum(0.0, time - delay)
    t_eff_pl = np.maximum(t_eff, 1.0)
    P_t = Jmin + (P0 - Jmin) * np.exp(-k1 * t_eff) * (t_eff_pl ** (-k2))
    flux, _ = simulate_membrane_flux(P_t, geom, D, dt, n_steps)
    flux_norm = flux / flux[0]
    flux_interp = np.interp(t_exp, time, flux_norm)
    return np.sum(((flux_interp - f_exp_norm) / f_exp_norm) ** 2)


# ============================================================
# 5. Optimization wrapper (now receives geom + time setup)
# ============================================================
def optimize_model(model_name, geom, time, dt, n_steps,
                   n_trials=10, verbose=True):
    best_cost = np.inf
    best_params = None

    for trial in range(n_trials):
        if model_name == "Exponential":
            init = np.array([0.4, 4e-5, 1e-3, 0.1 * 0.4 * 4e-5])
            lb = np.array([0.2, 3e-5, 5e-4, 0.05 * 3e-5])
            ub = np.array([1.0, 6e-5, 3e-3, 0.9 * 6e-5])

            fun = lambda p: cost_exponential(
                p, time, geom, t_exp, f_exp_norm, D, dt, n_steps, delay
            )

        elif model_name == "PowerLaw":
            init = np.array([0.4, 4e-5, 0.5, 0.1 * 0.4 * 4e-5])
            lb = np.array([0.2, 3e-5, 0.1, 0.05 * 3e-5])
            ub = np.array([1.0, 6e-5, 1.5, 0.9 * 6e-5])

            fun = lambda p: cost_powerlaw(
                p, time, geom, t_exp, f_exp_norm, D, dt, n_steps, delay
            )

        elif model_name == "Logistic":
            init = np.array([0.4, 4e-5, 0.002, 1200.0, 0.1 * 0.4 * 4e-5])
            lb = np.array([0.2, 3e-5, 1e-4, 500.0, 0.05 * 3e-5])
            ub = np.array([1.0, 6e-5, 0.01, 1500.0, 0.9 * 6e-5])

            fun = lambda p: cost_logistic(
                p, time, geom, t_exp, f_exp_norm, D, dt, n_steps
            )

        elif model_name == "Logistic_with_Jmin":
            init = np.array([
                np.random.rand() * 0.2,
                np.random.rand() * (6e-5 - 3e-5) + 3e-5,
                np.random.rand() * (5e-3 - 1e-5) + 1e-5,
                np.random.rand() * (1500.0 - 500.0) + 500.0
            ])
            lb = np.array([0.0, 3e-5, 1e-5, 500.0])
            ub = np.array([0.5, 6e-5, 5e-3, 1500.0])

            fun = lambda p: cost_logistic_jmin(
                p, time, geom, t_exp, f_exp_norm, D, dt, n_steps
            )

        elif model_name == "Hybrid_with_Jmin":
            init = np.array([
                np.random.rand() * 0.2,
                np.random.rand() * (6e-5 - 3e-5) + 3e-5,
                np.random.rand() * (1e-3 - 1e-6) + 1e-6,
                np.random.rand() * (0.5 - 0.01) + 0.01
            ])
            lb = np.array([0.0, 3e-5, 1e-6, 0.01])
            ub = np.array([0.5, 6e-5, 1e-3, 0.5])

            fun = lambda p: cost_hybrid_jmin(
                p, time, geom, t_exp, f_exp_norm, D, dt, n_steps, delay
            )

        else:
            raise ValueError(f"Unknown model name: {model_name}")

        bounds = list(zip(lb, ub))
        res = minimize(fun, x0=init, method="L-BFGS-B", bounds=bounds,
                       options={"maxiter": 1000, "ftol": 1e-9, "disp": False})

        if res.fun < best_cost:
            best_cost = res.fun
            best_params = res.x

        if verbose:
            print(f"  Trial {trial+1}/{n_trials}: cost={res.fun:.4e}")

    if verbose:
        print(f"Best cost for {model_name}: {best_cost:.4e}")
        print(f"Best params: {best_params}")

    return best_params, best_cost


# ============================================================
# 6. Main: takes geom_input (e.g. PCA geometry vector)
# ============================================================
def main(geom_input):
    # Build geometry and time grid from PCA geometry
    geom = build_geometry_from_pca(geom_input)
    time, dt, n_steps = build_time_grid(geom, D=D, T_total_min=150.0, safety=0.1)

    models = ["Exponential", "PowerLaw", "Logistic",
              "Logistic_with_Jmin", "Hybrid_with_Jmin"]

    results = {}

    for model in models:
        print(f"\n--- Optimizing {model} Model ---")
        best_params, best_cost = optimize_model(
            model, geom, time, dt, n_steps, n_trials=10, verbose=True
        )

        # Build P_t from best params
        if model == "Exponential":
            S0, P0, k, Pmin = best_params
            t_eff = np.maximum(0.0, time - delay)
            P_t = Pmin + (P0 - Pmin) * np.exp(-k * t_eff)

        elif model == "PowerLaw":
            S0, A, n, Pmin = best_params
            t_eff = np.maximum(time - delay, 1.0)
            P_t = Pmin + (A - Pmin) * (t_eff ** (-n))

        elif model == "Logistic":
            S0, P0, k, tmid, Pmin = best_params
            P_t = Pmin + (P0 - Pmin) / (1.0 + np.exp(k * (time - tmid)))

        elif model == "Logistic_with_Jmin":
            Jmin, P0, k, tmid = best_params
            P_t = Jmin + (P0 - Jmin) / (1.0 + np.exp(k * (time - tmid)))

        elif model == "Hybrid_with_Jmin":
            Jmin, P0, k1, k2 = best_params
            t_eff = np.maximum(0.0, time - delay)
            t_eff_pl = np.maximum(t_eff, 1.0)
            P_t = Jmin + (P0 - Jmin) * np.exp(-k1 * t_eff) * (t_eff_pl ** (-k2))

        flux, _ = simulate_membrane_flux(P_t, geom, D, dt, n_steps)
        flux_norm = flux / flux[0]
        flux_interp = np.interp(t_exp, time, flux_norm)

        y_true = f_exp_norm
        y_pred = flux_interp
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        R2 = 1.0 - ss_res / ss_tot

        results[model] = {
            "params": best_params,
            "flux": flux_norm,
            "flux_interp": flux_interp,
            "R2": R2
        }

    # ---- Plot comparison ----
    plt.figure()
    plt.plot(t_exp / 60.0, f_exp_norm, "ko", label="Experimental", markerfacecolor="k")
    colors = ["r-", "b-", "g-", "c-", "m-"]

    for color, model in zip(colors, models):
        flux_norm = results[model]["flux"]
        R2 = results[model]["R2"]
        plt.plot(time / 60.0, flux_norm, color,
                 label=f"{model} (R²={R2:.3f})")

    plt.xlabel("Time [min]")
    plt.ylabel("Normalized Flux")
    plt.grid(True)
    plt.legend()
    plt.title("All Models Comparison (Standard + Hybrid)")
    plt.tight_layout()
    plt.savefig("all_models_comparison_pcaGeom.png", dpi=200)
    plt.show()

    # ---- Exports ----
    out_dir = Path(".")
    T_export = pd.DataFrame({"Time_min": time / 60.0})
    for model in models:
        col_name = model.replace("_", "")
        T_export[col_name] = results[model]["flux"]
    ts_path = out_dir / "model_flux_timeseries_pcaGeom.xlsx"
    T_export.to_excel(ts_path, index=False)

    T_interp = pd.DataFrame({
        "Time_min_exp": t_exp / 60.0,
        "Experimental_Flux": f_exp_norm
    })
    for model in models:
        col_name = model.replace("_interp", "")
        T_interp[f"{col_name}_Interp"] = results[model]["flux_interp"]
    interp_path = out_dir / "model_flux_interp_vs_exp_pcaGeom.xlsx"
    T_interp.to_excel(interp_path, index=False)

    model_names = []
    R2_values = []
    param_cells = []

    for model in models:
        model_names.append(model)
        R2_values.append(results[model]["R2"])
        param_cells.append(results[model]["params"])

    T_summary = pd.DataFrame({
        "Model": model_names,
        "R_squared": R2_values,
        "Best_Params": param_cells
    })
    summary_path = out_dir / "model_summary_pcaGeom.xlsx"
    T_summary.to_excel(summary_path, index=False)

    print(f"\n✅ Exported:\n  {ts_path}\n  {interp_path}\n  {summary_path}")


if __name__ == "__main__":
    # Example: baseline geometry (no PCA perturbation)
    # Replace by your PCA vector, e.g. geom_input = pca_geom[i, :]
    geom_input = np.array([0.0, 0.0])
    main(geom_input)
